In [1]:
from stacksyms import newParseDWARF, parseDWARF, staticGDB

debugFilepath = 'data/cross-compile-dataset/bin/static/gcc/og/parallel'


In [3]:
module = newParseDWARF(debugFilepath)
firstComponent = module.children().__next__()
#print(firstComponent.types[5])
#print(firstComponent.variables)
functions = firstComponent.functions
functions = parseDWARF(debugFilepath)
functions = staticGDB(debugFilepath, functions)

01/22/2021 23:19:45 INFO:Processing compilation unit DWARF info (version 4)
01/22/2021 23:19:45 INFO:Processing function main.
01/22/2021 23:19:45 INFO:Found 17 stack elements.
01/22/2021 23:19:45 INFO:Processing function create_pipe_child.
01/22/2021 23:19:45 INFO:Found 4 stack elements.
01/22/2021 23:19:45 INFO:Processing function pipe_child.
01/22/2021 23:19:45 INFO:Found 5 stack elements.
01/22/2021 23:19:45 INFO:Processing function wait_for_child.
01/22/2021 23:19:45 INFO:Found 3 stack elements.
01/22/2021 23:19:45 INFO:Processing function exec_child.
01/22/2021 23:19:45 INFO:Found 6 stack elements.
01/22/2021 23:19:45 INFO:Processing function redirect.
01/22/2021 23:19:45 INFO:Found 3 stack elements.
01/22/2021 23:19:45 INFO:Processing function usage.
01/22/2021 23:19:45 INFO:Found 0 stack elements.
01/22/2021 23:19:45 INFO:Processing function printf.
01/22/2021 23:19:45 INFO:Found 1 stack elements.
01/22/2021 23:19:45 INFO:Processing function fprintf.
01/22/2021 23:19:45 INFO:Fo

NameError: name 'queries' is not defined

In [5]:
X, Y, Z = [], [], []
for func in functions:
    print('////////////////////////')
    print(func.name, hex(func.start), func.frame_base)
    for lvar in func.variables:
        print(lvar.name, lvar.type, "(bytesize = %d)"%lvar.type.byte_size)
        for loc in lvar.locations:
            print(hex(loc.begin) + " to " + hex(loc.end) + ": " \
            + str(loc.type)[13:]
            + str(loc.expr))
        print('')
#    X += [func] #generateFeature(func, functions)]
#    Y += [generateLabel(func, functions)]
#    Z += [generateDebugLabel(func, functions)]
#print([x+" => "+ str(y) for x,y in zip(X,Z)])



////////////////////////
main 0x400f88 Location(begin=0, end=0, type=<LocationType.STATIC_LOCAL: 1>, expr=(<ExprOp.CFA: 2>,))
maxjobs int (bytesize = 4)
0x400ff3 to 0x401099: STATIC_LOCAL(<ExprOp.CFA: 2>, -120, <ExprOp.ADD: 1>)
0x401099 to 0x4010b2: STATIC_LOCAL('rax',)
0x4010fc to 0x40123e: STATIC_LOCAL(<ExprOp.CFA: 2>, -120, <ExprOp.ADD: 1>)
0x401254 to 0x401289: STATIC_LOCAL(<ExprOp.CFA: 2>, -120, <ExprOp.ADD: 1>)
0x401289 to 0x40128b: STATIC_LOCAL('rax',)
0x40128b to 0x40154e: STATIC_LOCAL(<ExprOp.CFA: 2>, -120, <ExprOp.ADD: 1>)

curjobs int (bytesize = 4)
0x4013d8 to 0x401513: STATIC_LOCAL('rbx',)
0x40151d to 0x40154e: STATIC_LOCAL('rbx',)

maxload double (bytesize = 8)
0x401120 to 0x401178: STATIC_LOCAL('xmm0',)
0x401512 to 0x40151d: STATIC_LOCAL(<ExprOp.CFA: 2>, -128, <ExprOp.ADD: 1>)

argsatonce int (bytesize = 4)
0x400ff3 to 0x4011ab: STATIC_LOCAL(<ExprOp.CFA: 2>, -104, <ExprOp.ADD: 1>)
0x4011ab to 0x4011c8: STATIC_LOCAL('rax',)
0x4011c8 to 0x4013d8: STATIC_LOCAL(<ExprOp.CFA: 

TypeError: %d format: a number is required, not NoneType

In [ ]:
func   = 'quotearg_n_style_colon'
gdbmi  = GdbController()
result = gdbmi.write('file ' +debugFilepath)
result = gdbmi.write('info scope ' + func)
result = [msg['payload'] for msg in result if msg['type']=='console']
_collectLocals(result, functions[func])

In [ ]:
generateLabel(func, functions)

In [ ]:
cu = ELFFile(open(debugFilepath, mode='rb')).get_dwarf_info().iter_CUs().__next__()

[<Function: main@0x400f88>, <Function: create_pipe_child@0x400eab>, <Function: pipe_child@0x400b4e>, <Function: wait_for_child@0x400e32>, <Function: exec_child@0x400c49>, <Function: redirect@0x400aee>, <Function: usage@0x400c23>]
